## Ejercicios con Python Tarea Clase 19
### Pablo Ángel Mendoza Aguirre

Implementar el método de esquemas del trapecio, para valuar la opción call y put asiática con precio inicial, $S_0 = 100$, precio de ejercicio $K = 100$, tasa libre de riesgo $r = 0.10$, volatilidad $\sigma = 0.20$ y $T = 1$ año. Cuyo precio es $\approx 7.04$.

Observe que en esta tabla se encuentran los intervalos de confianza de la aproximación obtenida y además el tiempo de simulación que tarda en encontrar la respuesta cada método. 
- Se debe entonces realizar una simulación para la misma cantidad de trayectorias y número de pasos y construir una Dataframe de pandas para reportar todos los resultados obtenidos.**(70 puntos)**
- Compare los resultados obtenidos con los resultados arrojados por la función `Riemann_approach`. Concluya. **(30 puntos)**

# Riemman

In [ ]:
#importar los paquetes que se van a usar
import time,math 
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import datetime
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
%matplotlib inline
#algunas opciones para Pandas
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 78)
pd.set_option('precision', 3)

In [2]:
def BSprices(mu,sigma,S0,NbTraj,NbStep):
    T = 1
    nu = mu-(sigma**2)/2
    DeltaT = T/NbStep
    SqDeltaT = np.sqrt(DeltaT)
    #for i in range(NbStep):
    DeltaW = SqDeltaT*np.random.randn(NbTraj,NbStep-1)
    increments = nu*DeltaT + sigma*DeltaW
    concat = np.concatenate((np.log(S0)*np.ones([NbTraj,1]),increments),axis=1)
    LogSt = np.cumsum(concat,axis=1)
    St = np.exp(LogSt)
    t = np.arange(0,1,DeltaT)
    return St.T,t
def calc_daily_ret(closes):
    return np.log(closes/closes.shift(1)).iloc[1:]
def Riemann_approach_sc(K:'Strike price',r:'Tasa libre de riesgo',S0:'Precio inicial',
                     NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',
                     sigma:'Volatilidad'):
    # Resolvemos la ecuación de black scholes para obtener los precios
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    t = t*NbStep
    # Almacenamos los precios en un dataframe
    prices = pd.DataFrame(St,index=t)
    # Obtenemos los precios promedios
    Average_t = prices.expanding().mean()
    # Definimos el dataframe de strikes
    strike = pd.DataFrame(K*np.ones([NbStep,NbTraj]), index=t)
    # Calculamos el call de la opción según la formula obtenida para Sumas de Riemann
    call = pd.DataFrame({'Prima':np.exp(-r*T) \
                 *np.fmax(Average_t-strike,np.zeros([NbStep,NbTraj])).mean(axis=1)}, index=t)
    # intervalos de confianza
    confianza = 0.95
    sigma_est = call.sem().Prima
    mean_est = call.iloc[-1].Prima
    i1 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
#   return np.array([call.iloc[-1].Prima,i1[0],i1[1]])
    return call.iloc[-1].Prima
def Riemann_approach_sp(K:'Strike price',r:'Tasa libre de riesgo',S0:'Precio inicial',
                     NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',
                     sigma:'Volatilidad'):
    # Resolvemos la ecuación de black scholes para obtener los precios
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    t = t*NbStep
    # Almacenamos los precios en un dataframe
    prices = pd.DataFrame(St,index=t)
    # Obtenemos los precios promedios
    Average_t = prices.expanding().mean()
    # Definimos el dataframe de strikes
    strike = pd.DataFrame(K*np.ones([NbStep,NbTraj]), index=t)
    # Calculamos el call de la opción según la formula obtenida para Sumas de Riemann
    put = pd.DataFrame({'Prima':np.exp(-r*T) \
                 *np.fmax(strike-Average_t,np.zeros([NbStep,NbTraj])).mean(axis=1)}, index=t)
    # intervalos de confianza
    confianza = 0.95
    sigma_est = put.sem().Prima
    mean_est = put.iloc[-1].Prima
    i1 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
#   return np.array([call.iloc[-1].Prima,i1[0],i1[1]])
    return put.iloc[-1].Prima

## CALL sencillo pocas trayectorias

In [3]:
NbTraj = [1000,5000,10000]
NbStep = [10,50,100]

S0 = 100     # Precio inicial
r = 0.10     # Tasa libre de riesgo 
sigma = 0.2  # volatilidad
K = 100      # Strike price
T = 1        # Tiempo de cierre - años

# Call = np.zeros([len(NbTraj),len(NbStep)])
# intervalos = []#np.zeros([len(NbTraj),len(NbStep)])
M = list(map(lambda N_tra:list(map(lambda N_ste:Riemann_approach_sc(K,r,S0,N_tra,N_ste,sigma),
                                   NbStep)),
                                   NbTraj))
M = np.asmatrix(M)
# Visualización de datos 
filas = ['Nbtray = %i' %i for i in NbTraj]
col = ['NbStep = %i' %i for i in NbStep]
df = pd.DataFrame(index=filas,columns=col)
df.loc[:,:] = M
df

,NbStep = 10,NbStep = 50,NbStep = 100
Nbtray = 1000,6.471,6.492,7.229
Nbtray = 5000,6.489,6.842,7.194
Nbtray = 10000,6.366,6.906,7.064


## PUT sencillo pocas trayectorias

In [4]:
NbTraj = [1000,5000,10000]
NbStep = [10,50,100]

S0 = 100     # Precio inicial
r = 0.10     # Tasa libre de riesgo 
sigma = 0.2  # volatilidad
K = 100      # Strike price
T = 1        # Tiempo de cierre - años

# Call = np.zeros([len(NbTraj),len(NbStep)])
# intervalos = []#np.zeros([len(NbTraj),len(NbStep)])
M = list(map(lambda N_tra:list(map(lambda N_ste:Riemann_approach_sp(K,r,S0,N_tra,N_ste,sigma),
                                   NbStep)),
                                   NbTraj))
M = np.asmatrix(M)
# Visualización de datos 
filas = ['Nbtray = %i' %i for i in NbTraj]
col = ['NbStep = %i' %i for i in NbStep]
df = pd.DataFrame(index=filas,columns=col)
df.loc[:,:] = M
df

,NbStep = 10,NbStep = 50,NbStep = 100
Nbtray = 1000,2.374,2.288,2.229
Nbtray = 5000,2.232,2.338,2.447
Nbtray = 10000,2.187,2.275,2.270


In [5]:
def Riemann_approach_c(K:'Strike price',r:'Tasa libre de riesgo',S0:'Precio inicial',
                     NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',
                     sigma:'Volatilidad'):
    t1 = time.time()
    # Resolvemos la ecuación de black scholes para obtener los precios
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    t = t*NbStep
    # Almacenamos los precios en un dataframe
    prices = pd.DataFrame(St,index=t)
    # Obtenemos los precios promedios
    Average_t = prices.expanding().mean()
    # Definimos el dataframe de strikes
    strike = pd.DataFrame(K*np.ones([NbStep,NbTraj]), index=t)
    # Calculamos el call de la opción según la formula obtenida para Sumas de Riemann
    call = pd.DataFrame({'Prima':np.exp(-r*T) \
                 *np.fmax(Average_t-strike,np.zeros([NbStep,NbTraj])).mean(axis=1)}, index=t)
    # intervalos de confianza
    confianza = 0.95
    sigma_est = call.sem().Prima
    mean_est = call.iloc[-1].Prima
    i1 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
    t2 = time.time()
    speed = (t2-t1)
    return np.array([call.iloc[-1].Prima,i1[0],i1[1],speed]) 
def Riemann_approach_p(K:'Strike price',r:'Tasa libre de riesgo',S0:'Precio inicial',
                     NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',
                     sigma:'Volatilidad'):
    t1 = time.time()
    # Resolvemos la ecuación de black scholes para obtener los precios
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    t = t*NbStep
    # Almacenamos los precios en un dataframe
    prices = pd.DataFrame(St,index=t)
    # Obtenemos los precios promedios
    Average_t = prices.expanding().mean()
    # Definimos el dataframe de strikes
    strike = pd.DataFrame(K*np.ones([NbStep,NbTraj]), index=t)
    # Calculamos el call de la opción según la formula obtenida para Sumas de Riemann
    put = pd.DataFrame({'Prima':np.exp(-r*T) \
                 *np.fmax(strike-Average_t,np.zeros([NbStep,NbTraj])).mean(axis=1)}, index=t)
    # intervalos de confianza
    confianza = 0.95
    sigma_est = put.sem().Prima
    mean_est = put.iloc[-1].Prima
    i1 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
    t2 = time.time()
    speed = (t2-t1)
    return np.array([put.iloc[-1].Prima,i1[0],i1[1],speed])

## CALL

In [6]:
NbTraj = [1000,5000,10000,50000,100000,500000,1000000]#
NbStep = [10,50,100]
S0 = 100     # Precio inicial
r = 0.10     # Tasa libre de riesgo 
sigma = 0.2  # volatilidad
K = 100      # Strike price
T = 1        # Tiempo de cierre - años
speed = []
# Call = np.zeros([len(NbTraj),len(NbStep)])
# intervalos = []#np.zeros([len(NbTraj),len(NbStep)])
M = list(map(lambda N_tra:list(map(lambda N_ste:Riemann_approach_c(K,r,S0,N_tra,N_ste,sigma),NbStep)),NbTraj))

In [7]:
metodo = np.array(['1000','1000','1000','5000','5000','5000','10000','10000','10000','50000','50000','50000','100000','100000','100000','500000','500000','500000','1000000','1000000','1000000'])
steps = np.array(['10','50','100','10','50','100','10','50','100','10','50','100','10','50','100','10','50','100','10','50','100'])
aprox = np.array([M[0][0][0],M[0][1][0],M[0][2][0],M[1][0][0],M[1][1][0],M[1][2][0],M[2][0][0],M[2][1][0],M[2][2][0],M[3][0][0],M[3][1][0],M[3][2][0],M[4][0][0],M[4][1][0],M[4][2][0],M[5][0][0],M[5][1][0],M[5][2][0],M[6][0][0],M[6][1][0],M[6][2][0]])
Linf = np.array([M[0][0][1],M[0][1][1],M[0][2][1],M[1][0][1],M[1][1][1],M[1][2][1],M[2][0][1],M[2][1][1],M[2][2][1],M[3][0][1],M[3][1][1],M[3][2][1],M[4][0][1],M[4][1][1],M[4][2][1],M[5][0][1],M[5][1][1],M[5][2][1],M[6][0][1],M[6][1][1],M[6][2][1]])
Lsup = np.array([M[0][0][2],M[0][1][2],M[0][2][2],M[1][0][2],M[1][1][2],M[1][2][2],M[2][0][2],M[2][1][2],M[2][2][2],M[3][0][2],M[3][1][2],M[3][2][2],M[4][0][2],M[4][1][2],M[4][2][2],M[5][0][2],M[5][1][2],M[5][2][2],M[6][0][2],M[6][1][2],M[6][2][2]])
Ldiff = np.array(Lsup-Linf)
tiempos = np.array([M[0][0][3],M[0][1][3],M[0][2][3],M[1][0][3],M[1][1][3],M[1][2][3],M[2][0][3],M[2][1][3],M[2][2][3],M[3][0][3],M[3][1][3],M[3][2][3],M[4][0][3],M[4][1][3],M[4][2][3],M[5][0][3],M[5][1][3],M[5][2][3],M[6][0][3],M[6][1][3],M[6][2][3]])
tabla = pd.DataFrame(index=metodo,columns=['N. Pasos','Aprox','L. Inf','L. Sup','L. 95%','Tiempo (segs)'])
tabla.index.name = "Tray"
tabla['N. Pasos'] = steps
tabla['Aprox'] = aprox
tabla['L. Inf'] = Linf
tabla['L. Sup'] = Lsup
tabla['L. 95%'] = Ldiff
tabla['Tiempo (segs)'] = np.round(tiempos,2)
tabla

,N. Pasos,Aprox,L. Inf,L. Sup,L. 95%,Tiempo (segs)
Tray,,,,,,
1000,10,6.246,5.027,7.466,2.439,0.02
1000,50,6.981,6.450,7.512,1.063,0.03
1000,100,6.836,6.478,7.194,0.716,0.03
5000,10,6.217,4.994,7.441,2.448,0.08
5000,50,6.763,6.255,7.270,1.014,0.09
5000,100,6.944,6.586,7.302,0.717,0.12
10000,10,6.256,5.016,7.495,2.479,0.15
10000,50,6.918,6.402,7.435,1.033,0.16
10000,100,6.933,6.576,7.291,0.715,0.23


## PUT

In [8]:
NbTraj = [1000,5000,10000,50000,100000,500000,1000000]#
NbStep = [10,50,100]
S0 = 100     # Precio inicial
r = 0.10     # Tasa libre de riesgo 
sigma = 0.2  # volatilidad
K = 100      # Strike price
T = 1        # Tiempo de cierre - años
speed = []
# Call = np.zeros([len(NbTraj),len(NbStep)])
# intervalos = []#np.zeros([len(NbTraj),len(NbStep)])
M = list(map(lambda N_tra:list(map(lambda N_ste:Riemann_approach_p(K,r,S0,N_tra,N_ste,sigma),NbStep)),NbTraj))

In [9]:
metodo = np.array(['1000','1000','1000','5000','5000','5000','10000','10000','10000','50000','50000','50000','100000','100000','100000','500000','500000','500000','1000000','1000000','1000000'])
steps = np.array(['10','50','100','10','50','100','10','50','100','10','50','100','10','50','100','10','50','100','10','50','100'])
aprox = np.array([M[0][0][0],M[0][1][0],M[0][2][0],M[1][0][0],M[1][1][0],M[1][2][0],M[2][0][0],M[2][1][0],M[2][2][0],M[3][0][0],M[3][1][0],M[3][2][0],M[4][0][0],M[4][1][0],M[4][2][0],M[5][0][0],M[5][1][0],M[5][2][0],M[6][0][0],M[6][1][0],M[6][2][0]])
Linf = np.array([M[0][0][1],M[0][1][1],M[0][2][1],M[1][0][1],M[1][1][1],M[1][2][1],M[2][0][1],M[2][1][1],M[2][2][1],M[3][0][1],M[3][1][1],M[3][2][1],M[4][0][1],M[4][1][1],M[4][2][1],M[5][0][1],M[5][1][1],M[5][2][1],M[6][0][1],M[6][1][1],M[6][2][1]])
Lsup = np.array([M[0][0][2],M[0][1][2],M[0][2][2],M[1][0][2],M[1][1][2],M[1][2][2],M[2][0][2],M[2][1][2],M[2][2][2],M[3][0][2],M[3][1][2],M[3][2][2],M[4][0][2],M[4][1][2],M[4][2][2],M[5][0][2],M[5][1][2],M[5][2][2],M[6][0][2],M[6][1][2],M[6][2][2]])
Ldiff = np.array(Lsup-Linf)
tiempos = np.array([M[0][0][3],M[0][1][3],M[0][2][3],M[1][0][3],M[1][1][3],M[1][2][3],M[2][0][3],M[2][1][3],M[2][2][3],M[3][0][3],M[3][1][3],M[3][2][3],M[4][0][3],M[4][1][3],M[4][2][3],M[5][0][3],M[5][1][3],M[5][2][3],M[6][0][3],M[6][1][3],M[6][2][3]])
tabla = pd.DataFrame(index=metodo,columns=['N. Pasos','Aprox','L. Inf','L. Sup','L. 95%','Tiempo (segs)'])
tabla.index.name = "Tray"
tabla['N. Pasos'] = steps
tabla['Aprox'] = aprox
tabla['L. Inf'] = Linf
tabla['L. Sup'] = Lsup
tabla['L. 95%'] = Ldiff
tabla['Tiempo (segs)'] = np.round(tiempos,2)
tabla

,N. Pasos,Aprox,L. Inf,L. Sup,L. 95%,Tiempo (segs)
Tray,,,,,,
1000,10,2.464,1.985,2.944,0.959,0.02
1000,50,2.223,2.084,2.362,0.279,0.02
1000,100,2.385,2.287,2.484,0.197,0.03
5000,10,2.247,1.813,2.681,0.868,0.06
5000,50,2.383,2.231,2.536,0.305,0.08
5000,100,2.408,2.304,2.512,0.208,0.11
10000,10,2.207,1.781,2.632,0.851,0.15
10000,50,2.327,2.179,2.475,0.296,0.15
10000,100,2.365,2.264,2.466,0.202,0.22


# Trapecios

In [10]:
def TRAP_c(K:'Strike price',r:'Tasa libre de riesgo',S0:'Precio inicial',
                     NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',
                     sigma:'Volatilidad'):
    t1 = time.time()
    # Resolvemos la ecuación de black scholes para obtener los precios
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    t = t*NbStep
    # Almacenamos los precios en un dataframe
    prices = pd.DataFrame(St,index=t)
    # Obtenemos los precios promedios
    Average_t = prices.expanding().mean()
    # Definimos el dataframe de strikes
    strike = pd.DataFrame(K*np.ones([NbStep,NbTraj]), index=t)
    # Calculamos el call de la opción según la formula obtenida para Sumas de Riemann
    call = pd.DataFrame({'Prima':np.exp(-r*T) \
                 *np.fmax(Average_t-strike,np.zeros([NbStep,NbTraj])).mean(axis=1)}, index=t)
    # intervalos de confianza
    confianza = 0.95
    sigma_est = call.sem().Prima
    mean_est = call.iloc[-1].Prima
    i1 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
    t2 = time.time()
    speed = (t2-t1)
    return np.array([call.iloc[-1].Prima,i1[0],i1[1],speed]) 
def TRAP_p(K:'Strike price',r:'Tasa libre de riesgo',S0:'Precio inicial',
                     NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',
                     sigma:'Volatilidad'):
    t1 = time.time()
    # Resolvemos la ecuación de black scholes para obtener los precios
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    t = t*NbStep
    # Almacenamos los precios en un dataframe
    prices = pd.DataFrame(St,index=t)
    # Obtenemos los precios promedios
    Average_t = prices.expanding().mean()
    # Definimos el dataframe de strikes
    strike = pd.DataFrame(K*np.ones([NbStep,NbTraj]), index=t)
    # Calculamos el call de la opción según la formula obtenida para Sumas de Riemann
    put = pd.DataFrame({'Prima':np.exp(-r*T) \
                 *np.fmax(strike-Average_t,np.zeros([NbStep,NbTraj])).mean(axis=1)}, index=t)
    # intervalos de confianza
    confianza = 0.95
    sigma_est = put.sem().Prima
    mean_est = put.iloc[-1].Prima
    i1 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
    t2 = time.time()
    speed = (t2-t1)
    return np.array([put.iloc[-1].Prima,i1[0],i1[1],speed])

## Call

In [11]:
NbTraj = [1000,5000,10000,50000,100000,500000,1000000]#
NbStep = [10,50,100]
S0 = 100     # Precio inicial
r = 0.10     # Tasa libre de riesgo 
sigma = 0.2  # volatilidad
K = 100      # Strike price
T = 1        # Tiempo de cierre - años
speed = []
# Call = np.zeros([len(NbTraj),len(NbStep)])
# intervalos = []#np.zeros([len(NbTraj),len(NbStep)])
M = list(map(lambda N_tra:list(map(lambda N_ste:TRAP_c(K,r,S0,N_tra,N_ste,sigma),NbStep)),NbTraj))

In [12]:
metodo = np.array(['1000','1000','1000','5000','5000','5000','10000','10000','10000','50000','50000','50000','100000','100000','100000','500000','500000','500000','1000000','1000000','1000000'])
steps = np.array(['10','50','100','10','50','100','10','50','100','10','50','100','10','50','100','10','50','100','10','50','100'])
aprox = np.array([M[0][0][0],M[0][1][0],M[0][2][0],M[1][0][0],M[1][1][0],M[1][2][0],M[2][0][0],M[2][1][0],M[2][2][0],M[3][0][0],M[3][1][0],M[3][2][0],M[4][0][0],M[4][1][0],M[4][2][0],M[5][0][0],M[5][1][0],M[5][2][0],M[6][0][0],M[6][1][0],M[6][2][0]])
Linf = np.array([M[0][0][1],M[0][1][1],M[0][2][1],M[1][0][1],M[1][1][1],M[1][2][1],M[2][0][1],M[2][1][1],M[2][2][1],M[3][0][1],M[3][1][1],M[3][2][1],M[4][0][1],M[4][1][1],M[4][2][1],M[5][0][1],M[5][1][1],M[5][2][1],M[6][0][1],M[6][1][1],M[6][2][1]])
Lsup = np.array([M[0][0][2],M[0][1][2],M[0][2][2],M[1][0][2],M[1][1][2],M[1][2][2],M[2][0][2],M[2][1][2],M[2][2][2],M[3][0][2],M[3][1][2],M[3][2][2],M[4][0][2],M[4][1][2],M[4][2][2],M[5][0][2],M[5][1][2],M[5][2][2],M[6][0][2],M[6][1][2],M[6][2][2]])
Ldiff = np.array(Lsup-Linf)
tiempos = np.array([M[0][0][3],M[0][1][3],M[0][2][3],M[1][0][3],M[1][1][3],M[1][2][3],M[2][0][3],M[2][1][3],M[2][2][3],M[3][0][3],M[3][1][3],M[3][2][3],M[4][0][3],M[4][1][3],M[4][2][3],M[5][0][3],M[5][1][3],M[5][2][3],M[6][0][3],M[6][1][3],M[6][2][3]])
tabla = pd.DataFrame(index=metodo,columns=['N. Pasos','Aprox','L. Inf','L. Sup','L. 95%','Tiempo (segs)'])
tabla.index.name = "Tray"
tabla['N. Pasos'] = steps
tabla['Aprox'] = aprox
tabla['L. Inf'] = Linf
tabla['L. Sup'] = Lsup
tabla['L. 95%'] = Ldiff
tabla['Tiempo (segs)'] = np.round(tiempos,2)
tabla

,N. Pasos,Aprox,L. Inf,L. Sup,L. 95%,Tiempo (segs)
Tray,,,,,,
1000,10,6.348,5.103,7.593,2.490,0.02
1000,50,7.329,6.777,7.882,1.106,0.02
1000,100,6.708,6.371,7.045,0.674,0.03
5000,10,6.576,5.269,7.883,2.614,0.06
5000,50,7.015,6.490,7.539,1.050,0.08
5000,100,7.011,6.649,7.373,0.724,0.11
10000,10,6.451,5.168,7.734,2.566,0.13
10000,50,6.902,6.387,7.418,1.031,0.15
10000,100,7.007,6.644,7.369,0.725,0.22


## Put

In [13]:
NbTraj = [1000,5000,10000,50000,100000,500000,1000000]#
NbStep = [10,50,100]
S0 = 100     # Precio inicial
r = 0.10     # Tasa libre de riesgo 
sigma = 0.2  # volatilidad
K = 100      # Strike price
T = 1        # Tiempo de cierre - años
speed = []
# Call = np.zeros([len(NbTraj),len(NbStep)])
# intervalos = []#np.zeros([len(NbTraj),len(NbStep)])
M = list(map(lambda N_tra:list(map(lambda N_ste:TRAP_p(K,r,S0,N_tra,N_ste,sigma),NbStep)),NbTraj))

In [14]:
metodo = np.array(['1000','1000','1000','5000','5000','5000','10000','10000','10000','50000','50000','50000','100000','100000','100000','500000','500000','500000','1000000','1000000','1000000'])
steps = np.array(['10','50','100','10','50','100','10','50','100','10','50','100','10','50','100','10','50','100','10','50','100'])
aprox = np.array([M[0][0][0],M[0][1][0],M[0][2][0],M[1][0][0],M[1][1][0],M[1][2][0],M[2][0][0],M[2][1][0],M[2][2][0],M[3][0][0],M[3][1][0],M[3][2][0],M[4][0][0],M[4][1][0],M[4][2][0],M[5][0][0],M[5][1][0],M[5][2][0],M[6][0][0],M[6][1][0],M[6][2][0]])
Linf = np.array([M[0][0][1],M[0][1][1],M[0][2][1],M[1][0][1],M[1][1][1],M[1][2][1],M[2][0][1],M[2][1][1],M[2][2][1],M[3][0][1],M[3][1][1],M[3][2][1],M[4][0][1],M[4][1][1],M[4][2][1],M[5][0][1],M[5][1][1],M[5][2][1],M[6][0][1],M[6][1][1],M[6][2][1]])
Lsup = np.array([M[0][0][2],M[0][1][2],M[0][2][2],M[1][0][2],M[1][1][2],M[1][2][2],M[2][0][2],M[2][1][2],M[2][2][2],M[3][0][2],M[3][1][2],M[3][2][2],M[4][0][2],M[4][1][2],M[4][2][2],M[5][0][2],M[5][1][2],M[5][2][2],M[6][0][2],M[6][1][2],M[6][2][2]])
Ldiff = np.array(Lsup-Linf)
tiempos = np.array([M[0][0][3],M[0][1][3],M[0][2][3],M[1][0][3],M[1][1][3],M[1][2][3],M[2][0][3],M[2][1][3],M[2][2][3],M[3][0][3],M[3][1][3],M[3][2][3],M[4][0][3],M[4][1][3],M[4][2][3],M[5][0][3],M[5][1][3],M[5][2][3],M[6][0][3],M[6][1][3],M[6][2][3]])
tabla = pd.DataFrame(index=metodo,columns=['N. Pasos','Aprox','L. Inf','L. Sup','L. 95%','Tiempo (segs)'])
tabla.index.name = "Tray"
tabla['N. Pasos'] = steps
tabla['Aprox'] = aprox
tabla['L. Inf'] = Linf
tabla['L. Sup'] = Lsup
tabla['L. 95%'] = Ldiff
tabla['Tiempo (segs)'] = np.round(tiempos,2)
tabla

,N. Pasos,Aprox,L. Inf,L. Sup,L. 95%,Tiempo (segs)
Tray,,,,,,
1000,10,2.191,1.766,2.616,0.850,0.02
1000,50,2.295,2.147,2.443,0.296,0.02
1000,100,2.544,2.432,2.657,0.224,0.03
5000,10,2.332,1.886,2.779,0.893,0.07
5000,50,2.386,2.232,2.540,0.308,0.09
5000,100,2.335,2.234,2.436,0.201,0.11
10000,10,2.254,1.823,2.685,0.862,0.15
10000,50,2.318,2.172,2.465,0.294,0.16
10000,100,2.315,2.217,2.413,0.196,0.23
